# Mini-projet géophysique : interpolation de données manquantes

*Contexte* : dans le cadre d'acquisition de données sismiques, des capteurs (récepteurs) sont positionnés le long d'une ligne à la surface de la terre. Chacun enregistre au cours du temps le déplacement des particules (signal oscillant) qui varie lors du passage d'une onde acoustique ou élastique. Cet enregistrement s'appelle une **trace**. C'est une colonne dans les représentation graphiques plus bas.

En pratique, il arrive que certains capteurs soient défaillants ou encore qu'ils ne soient pas présents dans certaines zones (pour des problèmes d'accessibilité par exemple). L'objectif principal du mini-projet est de reconstituer les traces manquantes.

*Hypothèses* : nous ferons l'hypothèse que les **signaux sont essentiellement linéraires** dans les enregistrements (i.e. on peut localement déduire la trace suivante à partir de la trace précédente, avec un décalage en temps qui est proportionnel à la distance entre les traces). L'approximation linéaire est explicitée plus bas.

*Notions abordées* : 
* Lien entre la représentation avant et après transformée de Fourier 2d
* Effets de bords

Pour cela, nous considérons l'approche suivante :
* Transformée de Fourier 2d
* Identification des événements principaux
* Définition d'un masque
* Reconstitution des traces manquantes par transformee de Fourier inverse

L'approche est appliquée sur 3 jeux de données : 
* Donnees simples (un seul événement) : `gpanel1.npy`
* Donnees complexes (4 événements) : `gpanel2.npy`
* Sur vos propres données.

*Attentes* :
* Regard critique sur les résulats obtenus
* Inventivité (pour la troisième application)

*Consignes* :
* **Très important**: pour toutes les figures, bien indiquer les axes avec les bonnes unités et des labels lisibles

In [ ]:
# Modules
import numpy as np
import matplotlib.pyplot as plt

: 

**Lecture des données**

In [ ]:
# Labelsize (display)
labelsize   = 14
    
# Load the simple input data
# Input data (with missing traces)
gpanel1 = np.load('gpanel1.npy')
# Dense data (with all traces, only for comparison)
panel_dense1 = np.load('panel_dense1.npy')
nt, nx = gpanel1.shape
print("Taille des données :",nt,nx)

# Missing traces (index number)
mtr = [2,5,6,11,12,20,21,22,23]

# Vertical axis -- time
dt   = 3.125e-3  # increment (s)
at   = np.linspace(0,dt*(nt-1),nt)
# Horizontal axis -- space
dx   = 10. # increment (m)
ax   = np.linspace(0,dx*(nx-1),nx)

: 

**Affichage des données**

In [ ]:
# Display of the input data
vmax    = np.max(np.abs(gpanel1))
fig = plt.figure()
av = plt.subplot(111)
plt.imshow(gpanel1,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.title('Observables (avec traces manquantes, mises à zero)', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([-vmax,vmax])
plt.set_cmap('bwr')

fig = plt.figure()
av = plt.subplot(111)
plt.imshow(panel_dense1,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.title('Observables non disponibles (seulement pour comparaison)', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([-vmax,vmax])
plt.set_cmap('bwr')

: 

# Partie 1. Données simples (7 points)

## Approche théorique

On peut supposer que $$u(t,x) = S\big(t-t_0-p_0(x-x_0)\big)$$

C'est l'approximation linéaire mentionnée plus haut,
avec $S(t)$ une ondelette source, $t_0$ le temps à la position $x=x_0$, et la pente_tab_tab_tab $p_0$ en s/m associée à l'événement. Ici : $t_0=0.2$ s (pour $x_0 = 0$ m) et $p_0=0.002$ s/m.

Tracer la droite définie par $t_0$ et $p_0$ sur les observables (`gpanel1`) pour vérifier qu'elle se superpose bien à l'événement principal.


In [ ]:
vmax = np.max(np.abs(gpanel1))
fig = plt.figure()
av = plt.subplot(111)
plt.imshow(gpanel1,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.title('Observables (avec traces manquantes, mises à zero)', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([-vmax,vmax])
plt.set_cmap('bwr')

#Tracer la droite 
t0 = 0.2
x0 = 0 
p0 = 0.002
x_range = np.array([ax[0], ax[-1]])  # valeurs x pour tracer la droite
t_line = t0 + p0 * (x_range - x0)  

plt.plot(x_range, t_line, color='green', linewidth=2)
plt.show()

: 

On observe que la droite tracée en vert et le signal enregistré se superposent bien. 


Calculer l'expression de la transformée de Fourier 2d $\hat{u}(\omega,k)$ de $u(t,x)$ à partir de la transformée de Fourier 1d $\hat{S}(\omega)$ de $S(t)$. En déduire une expression simple de $|\hat{u}|$ en lien avec $|\hat{S}(\omega)|$. Expliquer toujours à partir des équations pourquoi on s'attend à avoir un événement linéaire dans le domaine de Fourier. Pourquoi l'événement passe-t-il par l'origine $(0,0)$ dans le domaine de Fourier? Donner une expression quantitative entre la pente_tab dans le domaine de Fourier $(\omega,k)$ et la pente dans le domaine $(t,x)$.

Pour simplifier l'expression, on peut utiliser

$$\int dx e^{-i k x} = \delta(k)$$

avec $\delta$ la distribution de Dirac.

On utilise la définition de la transformée de Fourier, $$ \^u(\omega,k)=\int_{x=-\infty}^{x=+\infty} \int_{t=-\infty}^{t=+\infty} e^{-i(\omega t + k x)} u(t,x) dt dx = \int_{x=-\infty}^{x=+\infty} \int_{t=-\infty}^{t=+\infty} e^{-i(\omega t + k x)} S(t-t_{0}-p_{0}(x-x_{0})) dt dx $$ 
Soit  $$ \^u(\omega,k)= \int_{x=-\infty}^{x=+\infty}   e^{-ikx} \int_{t=-\infty}^{t=+\infty} e^{-i( \omega t)} S(t-t_{0}-p_{0}(x-x_{0})) dt dx $$. 
On pose le changement de variable, $$ v = t-t_{0}-p_{0}(x-x_{0}) $$. 
Alors, d'après le théorème de Fubini, $$ \^u(\omega,k) =  \int_{x=-\infty}^{x=+\infty}   e^{-ikx}  dx \int_{v=-\infty}^{v=+\infty} e^{-i\omega (v +t_{0} + p_{0}(x-x_{0})) }  S(v) dv $$. On a $$ \^u(\omega,k) = e^{-i\omega (t_{0}-p_{0} x_{0})} \int_{x=-\infty}^{x=+\infty}   e^{-i(k+\omega p_{0}) x} dx \int_{v=-\infty}^{v=+\infty} S(v) e^{-i\omega v}) $$. 

On en déduit alors que $$  \^u(\omega,k) = e^{-i \omega(t_{0}-p_{0}x_{0})}  \delta(k+\omega p_{0}) \^S(\omega) $$ et que $ |\^u(\omega,k) | = | \delta(k+\omega p_{0})  \^S(\omega)| $.

Alors, $ \^u(\omega=0,k=0)= \^S(0) $ et puisque $\^S(0)=\int_{t=-\infty}^{t=+\infty} S(t) dt $ l'événement passe par (0,0) dans le domaine de Fourier car il s'agit de l'intégrale du signal en un point. D'après l'expression obtenue par le calcul, la transformée de Fourier est non-nulle si $ k+\omega p_{0}=0$ donc on a  $ p_{0} = \frac{-k}{\omega}$.

## Implémentation
Représenter la transformée de Fourier 2d et bien préciser les axes. Faire le lien entre l'événement dans l'espace $(t,x)$ et celui dans l'espace de Fourier. Comme pour le domaine $(t,x)$, tracer la droite correspondante dans le domaine $(\omega,k)$. Expliquer ce qu'il se passe aux bords dans le domaine de Fourier. N'hésitez pas à zoomer autour de la position centrale $(0,0)$ dans le domaine de Fourier pour une meilleure représentation.

In [ ]:
tf = np.fft.fftshift(np.fft.fft2(gpanel1))
# fftshift est utilisé pour recentrer les fréquences à zéro

#renormalisation de la fréquence et du vecteur d'onde
n = gpanel1.shape[0]
m = gpanel1.shape[1]
dt = np.abs(at[1] - at[0])
dx = np.abs(ax[1] - ax[0])
dw = 2 * np.pi / (m * dt)
dk = 2 * np.pi / (n * dx)
tf = np.fft.fftshift(np.fft.fft2(gpanel1))
w = np.fft.fftshift(np.fft.fftfreq(m, dt)) * 2 * np.pi
k = np.fft.fftshift(np.fft.fftfreq(n, dx)) * 2 * np.pi

# Tracer la transformée de Fourier 2D
plt.imshow(np.abs(tf), extent=[k[0],k[-1],w[0],w[-1]], aspect='auto')
plt.title('Transformée de Fourier 2D')
plt.xlabel('Ondelette k (en m-1)')
plt.ylabel('Fréquence w (en s-1)')

plt.ylim((-250,250)) #afin d'améliorer l'affichage 
plt.colorbar(label='Amplitude')
plt.xlim(k[0],k[-1]) # Zoom (assez violent) pour mieux voir le signal de l'événement 
plt.ylim(w[0],w[-1])
pente_droite=450
plt.plot(k,k*pente_droite,c='green')
plt.legend()
plt.show()

: 

On observe que le lien entre les ondelettes $k$ et les pulsations $\omega$ est tel que $ \omega =\frac{k}{0.002} $. 

## Définition d'un masque dans le domaine de Fourier 2d
On note $\omega$ la fréquence angulaire et $k$ le nombre d'onde. Dans le domaine de Fourier 2d, on souhaite définir un masque avec des 1 qui couvrent les événements principaux. A partir de $(\omega,k)=(0,0)$, faire des sommations selon différentes pente_tabs (dans le domaine de Fourier, après avoir pris le module des valeurs complexes) et tracer la valeur de ces sommes en fonction de la pente_tab. On s'attend à ce que une valeur ressorte. Calculer une fonction qui sélectionne automatiquement cette valeur et construire un masque dans le domaine de Fourier 2d qui vaut 1 autour de la valeur principale et 0 en dehors.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n = gpanel1.shape[0]
m = gpanel1.shape[1]
dt = np.abs(at[1] - at[0])
dx = np.abs(ax[1] - ax[0])
dw = 2 * np.pi / (m * dt)
dk = 2 * np.pi / (n * dx)
tf = np.fft.fftshift(np.fft.fft2(gpanel1))
w = np.fft.fftshift(np.fft.fftfreq(m, dt)) * 2 * np.pi
k = np.fft.fftshift(np.fft.fftfreq(n, dx)) * 2 * np.pi

def contin(tf,k_,w_):
    return tf[int((w_-min(w))/(max(w)-min(w))*tf.shape[0]),int((k_-min(k))/(max(k)-min(k))*tf.shape[1])] # On récupère la valeur de la transformée de Fourier pour (k,w): il faut calculer les indices 

def pente_():
    pente=np.linspace(-600,-400,100) # On choisit une liste assez large de pentes possibles 
    somme_pente=[]
    for p in pente:
        w_selec=[(k_,k_*p) for k_ in k[1:-1] if (k_*p > min(w)) and (k_*p < max(w))] # On génère des couples (k,pente*k), en prenant garde à ce que l'ordonnée reste dans l'intervalle admissible
        s=0
        for i in range(len(w_selec)):
            s+=np.abs(contin(tf,w_selec[i][0],w_selec[i][1])) # On fait la somme le long de la droite donnée 
        somme_pente.append(s)
    return pente,somme_pente,pente[np.argmax(somme_pente)]

print("On trouve que la pente qui maximise la somme le long de la droite est",pente_()[2])

plt.title("Somme des valeurs de la transformée le long d'une droite (en fonction de la pente)")
plt.xlabel("pente p (en m.s-1)")
plt.ylabel("Somme le long de la droite ")
plt.plot(pente_()[0],pente_()[1])
plt.show()
# Tracer la transformée de Fourier 2D
plt.figure()
plt.imshow(np.abs(tf), extent=[k[0], k[-1], w[0],w[-1]], aspect='auto')
plt.title('Transformée de Fourier 2D')
plt.plot(k,444*k)
plt.xlabel('Ondelette k (en m-1)')
plt.ylabel('Fréquence w (en s-1)')
plt.colorbar(label='Amplitude')
plt.legend()
plt.show()


: 

## Application du masque et analyse des résultats
Appliquer le masque (sur les données après transformée de Fourier 2d, sans prendre le module des valeurs), puis appliquer la transformée de Fourier inverse. Commenter les résultats. Si nécessaire, revenir à la question précédente pour changer la forme du masque et avoir une meilleure interpolation des données. Expliquer ces changements. Discuter de l'importance du masque.

In [ ]:
""" La valeur maximale de la somme est atteinte pour une pente de p=200 s.m-1. On applique donc le filtre avec cette valeur de pente."""

n = gpanel1.shape[0]
dt = np.abs(at[1] - at[0])
dx = np.abs(ax[1] - ax[0])
dw = 2 * np.pi / (n * dt)
dk = 2 * np.pi / (n * dx)
tf = np.fft.fftshift(np.fft.fft2(gpanel1))
w = np.fft.fftshift(np.fft.fftfreq(n, dt)) * 2 * np.pi
k = np.fft.fftshift(np.fft.fftfreq(n, dx)) * 2 * np.pi

plt.figure()
plt.imshow(np.abs(tf), extent=[k[0], k[-1], -250, 250], aspect='auto')
plt.title('Transformée de Fourier 2D avant masque')
plt.xlabel('Ondelette k (en m-1)')
plt.ylabel('Fréquence w (en s-1)')
plt.colorbar(label='Amplitude')
plt.legend()
plt.show()


p_=-444
tf_m=tf.copy()

# On crée le masque adapté, en mettant des 1 qu'au niveau des indices qui correspondent aux points le long de la droite. 
mask=np.zeros((tf.shape[0],tf.shape[1]),dtype=int)
for k_ in np.linspace(-0.25,0.25,100):
    mask[int((k_*p_-min(w))/(max(w)-min(w))*tf.shape[0]),int((k_-min(k))/(max(k)-min(k))*tf.shape[1])]=1
    if int((k_*p_-min(w))/(max(w)-min(w))*tf.shape[0]) < tf.shape[0]-2:
        mask[int((k_*p_-min(w))/(max(w)-min(w))*tf.shape[0])+1,int((k_-min(k))/(max(k)-min(k))*tf.shape[1])]=1
    if int((k_*p_-min(w))/(max(w)-min(w))*tf.shape[0])-1>0:
        mask[int((k_*p_-min(w))/(max(w)-min(w))*tf.shape[0])-1,int((k_-min(k))/(max(k)-min(k))*tf.shape[1])]=1
    

tf_m=tf*mask
# Tracer la transformée de Fourier 2D
plt.figure()
plt.imshow(np.abs(tf_m), extent=[k[0], k[-1], w[0], w[-1]], aspect='auto')
plt.title('Transformée de Fourier 2D après masque ')
plt.xlabel('Ondelette k (en m-1)')
plt.ylabel('Fréquence w (en s-1)')
plt.colorbar(label='Amplitude')
plt.legend()
plt.show()

: 

In [ ]:
vmax=np.max(np.real(np.fft.ifft2(tf_m)))
fig = plt.figure()
av = plt.subplot(111)
plt.imshow(np.real(np.fft.ifft2(tf_m)),extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.title('Résultat après traitement du signal', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([0,vmax])
plt.set_cmap('bwr')

: 

On constate que le traitement appliqué permet de faire disparaître les zones manquantes suite à l'acquisition du signal. Néanmoins, on constate l'apparition de nouveaux signaux parasites. On peut proposer, pour résoudre le problème, d'appliquer un filtre passe_bas aux vecteurs d'onde k. 

# Partie 2. Données complexes (7 points)

Recommencer la même procédure (interpolation des données) sur les données plus complexes (`gpanel2.npy`). Analyser les similitudes et différences, et en particulier les difficultés rencontrées.

Les pente sont : 
$p_0 = 0.006$, $p_1 = 0.002$, $p_2 = -0.004$ et $p_3 = -0.0005$ en s/m 
et les temps associes (pour $x_0=0$ m) : $t_0 = 0.1$, $t_1 = 0.3$, $t_2 = 1.1$ et $t_3 = 1.8$ en s.


In [ ]:
# Load the input data
# Input data (with missing traces)
gpanel2 = np.load('gpanel2.npy')
# Dense data (with all traces, only for comparison)
panel_dense2 = np.load('panel_dense2.npy')
nt, nx = gpanel2.shape
print("Taille des données :",nt,nx)

: 

**Affichage des données**

In [ ]:
# Display of the input data
vmax    = np.max(np.abs(gpanel2))
fig = plt.figure()
av = plt.subplot(111)
plt.imshow(gpanel2,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.title('Observables (avec traces manquantes, mises à zero)', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([-vmax,vmax])
plt.set_cmap('bwr')

fig = plt.figure()
av = plt.subplot(111)
plt.imshow(panel_dense2,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.title('Observables non disponibles (seulement pour comparaison)', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([-vmax,vmax])
plt.set_cmap('bwr')

: 

In [ ]:
# Calcul de la transformée de Fourier 2D
tf2 = np.fft.fftshift(np.fft.fft2(gpanel2))
# fftshift est utilisé pour recentrer les fréquences à zéro

# Création des axes de fréquence (w) et d'onde (k)
n = gpanel2.shape[0]
m = gpanel2.shape[1]
dt = np.abs(at[1] - at[0])
dx = np.abs(ax[1] - ax[0])
dw = 2 * np.pi / (m * dt)
dk = 2 * np.pi / (n * dx)
w2 = np.fft.fftshift(np.fft.fftfreq(m, dt)) * 2 * np.pi
k2 = np.fft.fftshift(np.fft.fftfreq(n, dx)) * 2 * np.pi

# Tracer la transformée de Fourier 2D
plt.imshow(np.abs(tf2), extent=[k2[0], k2[-1], w2[0], w2[-1] ], aspect='auto')
plt.title('Transformée de Fourier 2D du signal gpanel2')
plt.xlabel('Ondelette k (en m-1)')
plt.ylabel('Fréquence w (en s-1)')
plt.ylim(-100,100)
plt.colorbar(label='Amplitude')
plt.legend()
plt.show()

: 

On se contente ici d'adapter la méthode de la partie 1. La difficulté principale est qu'il faut trouver plusieurs pentes, et plus seulement la pente qui maximise la somme le long de la droite. On constate cependant que 4 droites (2 avec une pente négative, et 2 avec une pente positive) semblent maximiser la somme du module de la transformée de Fourier le long de la droite. On cherche donc quatre extrema de la somme de la transformée de Fourier en tenant compte de cette information. 

In [1]:

mask=np.zeros((tf2.shape[0],tf2.shape[1]),dtype=int)
tf2_m=np.zeros((tf2.shape[0],tf2.shape[1]),dtype='complex')
for p_ in [1/0.006, 1/0.002 , 1/(-0.004) , 1/(-0.0005)]:
    for k_ in np.linspace(-0.25,0.25,100):
        mask[int((k_*p_-min(w))/(max(w)-min(w))*tf2.shape[0]),int((k_-min(k))/(max(k)-min(k))*tf2.shape[1])]=1        
        if int((k_*p_-min(w))/(max(w)-min(w))*tf2.shape[0]) < tf2.shape[0]-2:
            mask[int((k_*p_-min(w))/(max(w)-min(w))*tf2.shape[0])+1,int((k_-min(k))/(max(k)-min(k))*tf2.shape[1])]=1
        if int((k_*p_-min(w))/(max(w)-min(w))*tf2.shape[0])-1>0:
            mask[int((k_*p_-min(w))/(max(w)-min(w))*tf2.shape[0])-1,int((k_-min(k))/(max(k)-min(k))*tf2.shape[1])]=1


tf2_m=tf2*mask

# Tracer la transformée de Fourier 2D
plt.imshow(np.abs(tf2_m), extent=[k2[0], k2[-1], w2[0], w2[-1] ], aspect='auto')
plt.title('Transformée de Fourier 2D du signal gpanel2 après traitement (avec un masque adapté)')
plt.xlabel('Ondelette k (en m-1)')
plt.ylabel('Fréquence w (en s-1)')
plt.ylim(-100,100)
plt.colorbar(label='Amplitude')
plt.legend()
plt.show()

NameError: name 'np' is not defined

Cependant, il est clair que les fenêtres appliquées successivemebt se chevauchent. Cela conduit à réduire voire annuler certaines composantes, qu'on aurait pourtant voulu garder, de la transformée de Fourier du signal.

In [ ]:
vmax=np.max(np.real(np.fft.ifft2(tf2_m)))
fig = plt.figure()
av = plt.subplot(111)
plt.imshow(np.real(np.fft.ifft2(tf2_m)),extent=[ax[0],ax[-1],at[-1],at[0]], aspect='auto')
plt.title('Résultat après traitement du signal', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([0,vmax])
plt.set_cmap('bwr')

: 

On constate que l'on ne peut plus distinguer clairement les 4 droites, qui étaient pourtant clairement identifiables à l'origine. On va donc chercher à modifier le traitement appliqué à la transformée de Fourier de gpanel2. 

In [ ]:
mask=np.zeros((tf2.shape[0],tf2.shape[1]),dtype=int)
tf2_m=np.zeros((tf2.shape[0],tf2.shape[1]),dtype='complex')
for p_ in [1/0.006, 1/0.002 , 1/(-0.004) , 1/(-0.0005)]:
    for k_ in np.linspace(-0.25,0.25,100):
        mask[int((k_*p_-min(w))/(max(w)-min(w))*tf2.shape[0]),int((k_-min(k))/(max(k)-min(k))*tf2.shape[1])]=1       

tf2_m=tf2*mask

va_abs_max=np.max(np.abs(tf2_m))
for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        if mask[i][j]:
            tf2_m[i][j]=np.sqrt(np.abs(va_abs_max*tf2_m[i][j]))/np.sqrt((np.abs(tf2_m[i][j])+0.001))
            #tf2_m[i][j]=np.sqrt((np.abs(tf2_m[i][j]))/(va_abs_max))*tf2_m[i][j]

plt.imshow(np.abs(tf2_m), extent=[k2[0], k2[-1], w2[0], w2[-1] ], aspect='auto')
plt.title('Transformée de Fourier 2D du signal gpanel2 après le nouveau traitement (avec un masque adapté)')
plt.xlabel('Ondelette k (en m-1)')
plt.ylabel('Fréquence w (en s-1)')
plt.ylim(-100,100)
plt.colorbar(label='Amplitude')
plt.legend()
plt.show()

: 

In [ ]:
vmax=np.max(np.real(np.fft.ifft2(tf2_m)))
fig = plt.figure()
av = plt.subplot(111)
plt.imshow(np.real(np.fft.ifft2(tf2_m)),extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.title('Résultat après traitement du signal', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([0,vmax])
plt.set_cmap('bwr')

: 

# Partie 3. Analyse des limites de l'approche (6 points)
Proposer une nouvelle application sur votre propre jeu de donnees. Vous pouvez par exemple :
* ajouter du bruit sur gpanel1 ou gpanel2
* enlever encore plus de données en entrée (plus de traces blanches) 
* tester sur vos propres données / sur des images récupérées sur internet

Illustrer et discuter une limite (ou plusieurs) que rencontre l'interpolation dans le domaine de Fourier.

**Cette dernière partie est très ouverte et je fais appel à votre créativité.**